This file is named `vec2txt` but really is just `jgm/ecog2txt`. I claim no code written in this file.

In [ ]:
# INIT
%reset -s -f
# %matplotlib ipympl
# %matplotlib inline 
# notebook
%load_ext autoreload
%autoreload 2

# standard modules    
import os
import numpy as np

# third-party packages
import torch

# local packages
from utils_jgm.machine_compatibility_utils import MachineCompatibilityUtils
import ecog2txt
import ecog2txt.trainers as e2t_trainers
from ecog2txt import makin_lab_data_generators

MCUs = MachineCompatibilityUtils()

In [ ]:
# CREATE A NEW MODEL
experiment = 'mocha-1'
token_type = 'word_sequence'
project = 'EFC'
subject_ids = [401]

experiment_manifest_name = os.path.join(
    project,
    '_'.join([experiment, token_type]) + '.yaml'
)
trainer = e2t_trainers.MultiSubjectTrainer(
    experiment_manifest_name=experiment_manifest_name,
    subject_ids=subject_ids,
    SN_kwargs={
        # 'Nepochs': 20,
        # 'training_GPUs': [1],
        'layer_sizes': {
            'decoder_embedding': [150],
            'decoder_projection': [],
            'decoder_rnn': [800],
            'encoder_embedding': [100],
            'encoder_rnn': [400, 400, 400],
            # 'encoder_0_projection': [225],
            'encoder_1_projection': [225],
        },
        'coupling': 'attention',
        'RNN_type': 'GRU',
    },
    DG_kwargs={
        'REFERENCE_BIPOLAR': True,
        'prefix': project,
        'subdir': 'with_audio',
        'num_cepstral_coeffs': 13,
        # 'max_seconds': 3,
        'tf_record_partial_path': os.path.join(
            MCUs.get_path('data'),
            'ecog2txt', token_type, 'tf_records_w2v_sil_inc', # 200 Hz, 429 channels
            # 'ecog2txt', token_type, 'tf_records_BY_raw_filtered', # 200 Hz, 238 channels
            project + repr(subject_ids[-1]) + '_B{0}.tfrecord', # 101 Hz
        ),
        'REFERENCE_BIPOLAR': False,
        'good_electrodes': list(range(512)),
        'grid_size': [512, 1],
        'sampling_rate': 101.7
    },
    ES_kwargs = {
        'data_mapping': {
            'encoder_inputs': 'ecog_sequence',
            'encoder_1_targets': 'phoneme_sequence',
            # 'encoder_1_targets': 'audio_sequence',
            'decoder_targets': 'text_sequence',
        },
    },
)

In [ ]:
print('Before:')
print(trainer.ecog_subjects[-1].block_ids)

trainer.ecog_subjects[-1].block_ids = {
    'validation': {73},
    'testing': {},
    'training': {69, 77, 83, 87}# {4, 41, 57, 61, 66, 69, 73, 77, 83, 87}
}

# trainer.ecog_subjects[-1].block_ids = {
#     'validation': {72},
#     'testing': {},
#     'training': {23,3}
# }

# ['testing'] = set()
# trainer.ecog_subjects[-1].block_ids['training'] = {41, 57, 61, 66, 69, 73, 77, 83, 87}
# trainer.ecog_subjects[-1].block_ids['validation'] = {4}

print('After:')
print(trainer.ecog_subjects[-1].block_ids)

In [ ]:
import torch
from machine_learning.neural_networks.torch_sequence_networks import (
    Sequence2Sequence, SequenceTrainer
)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
manifest = trainer.experiment_manifest[trainer.ecog_subjects[-1].subnet_id]

torch_trainer = SequenceTrainer(manifest, trainer.ecog_subjects)

In [ ]:
torch_trainer.train_and_assess(400, trainer.net, device)